In [1]:
import os
import json
import pathlib
from datetime import datetime
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import torch
from typing import List, Dict, Union, Tuple, NamedTuple
from tqdm import tqdm
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = f"models/xgb/{ts}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
num_boost_round: int = 100
lr: Tuple[float, float] = (1e-3, 1e-3)
feature_fraction: Tuple[float, float] = (1, 1)
min_data_in_leaf: Tuple[int, int] = (20, 20)
objective: str = "binary:logistic"
n_trials: int = 1
label = "generated"

In [3]:
df = pd.read_parquet("input/features.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 30506 entries, row_id to tf_Ġzyg
dtypes: float32(30490), int16(2), int32(5), int8(1), object(8)
memory usage: 25.2+ GB


In [4]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
print(f"{len(features)} features\n{features[:100]}")

30493 features
['ch_digit_frac', 'ch_len', 'ch_letter_frac', 'ch_punc_frac', 'ch_repeat_char_frac', 'ch_space_frac', 'ch_upper_frac', 'tf_0', 'tf_00', 'tf_000', 'tf_0000', 'tf_01', 'tf_1', 'tf_10', 'tf_100', 'tf_11', 'tf_12', 'tf_13', 'tf_14', 'tf_15', 'tf_16', 'tf_17', 'tf_18', 'tf_19', 'tf_2', 'tf_20', 'tf_200', 'tf_21', 'tf_22', 'tf_23', 'tf_24', 'tf_25', 'tf_26', 'tf_27', 'tf_28', 'tf_29', 'tf_3', 'tf_30', 'tf_31', 'tf_32', 'tf_33', 'tf_34', 'tf_35', 'tf_36', 'tf_37', 'tf_38', 'tf_380', 'tf_39', 'tf_4', 'tf_40', 'tf_41', 'tf_42', 'tf_43', 'tf_44', 'tf_45', 'tf_46', 'tf_47', 'tf_48', 'tf_49', 'tf_5', 'tf_50', 'tf_53', 'tf_55', 'tf_56', 'tf_57', 'tf_58', 'tf_6', 'tf_60', 'tf_65', 'tf_66', 'tf_67', 'tf_68', 'tf_7', 'tf_70', 'tf_75', 'tf_76', 'tf_77', 'tf_78', 'tf_8', 'tf_80', 'tf_84', 'tf_85', 'tf_86', 'tf_87', 'tf_88', 'tf_89', 'tf_9', 'tf_90', 'tf_92', 'tf_94', 'tf_95', 'tf_96', 'tf_97', 'tf_98', 'tf_99', 'tf_a', 'tf_aa', 'tf_aar', 'tf_ab', 'tf_aba']


In [5]:
#X_train, X_test, y_train, y_test = train_test_split(tra[features], tra[label], test_size=0.2)

th=0.23
tra = df[df["white_sim"]>=th]
val = df[df["white_sim"]<th]
t = len(tra)
v = len(val)
n = t+v
print(f"val%={v/n:.4f}, len(tra)={t:,}, len(val)={v:,}")
dtrain = xgb.DMatrix(tra[features], tra[label], enable_categorical=False)
dval = xgb.DMatrix(val[features], val[label], enable_categorical=False)
pdx.value_counts(val[label])

val%=0.0094, len(tra)=219,529, len(val)=2,083


,count,percent
generated,,
0,2077,0.99712
1,6,0.00288


In [6]:
%%time
model = xgb.train(
   params={
       "objective": objective,
       "learning_rate": 5e-2,
       "min_child_weight": 20,
       "colsample_bytree": 0.5,
       "max_depth": 6,
   },
   dtrain=dtrain,
   num_boost_round=2000,
   evals=[(dtrain, "train"), (dval, "val")],
   verbose_eval=40,
   early_stopping_rounds=100,
)
print(f"best score {model.best_score:.5f} at iteration {model.best_iteration}")
model.save_model(f"{job_dir}/model.json")

[0]	train-logloss:0.29270	val-logloss:0.16858
[40]	train-logloss:0.08731	val-logloss:0.03241
[80]	train-logloss:0.05195	val-logloss:0.01421
[120]	train-logloss:0.03891	val-logloss:0.01138
[160]	train-logloss:0.03185	val-logloss:0.01057
[200]	train-logloss:0.02644	val-logloss:0.00981
[240]	train-logloss:0.02285	val-logloss:0.00924
[280]	train-logloss:0.02034	val-logloss:0.00867
[320]	train-logloss:0.01833	val-logloss:0.00831
[360]	train-logloss:0.01672	val-logloss:0.00787
[400]	train-logloss:0.01539	val-logloss:0.00773
[440]	train-logloss:0.01422	val-logloss:0.00748
[480]	train-logloss:0.01322	val-logloss:0.00727
[520]	train-logloss:0.01235	val-logloss:0.00708
[560]	train-logloss:0.01153	val-logloss:0.00700
[600]	train-logloss:0.01082	val-logloss:0.00686
[640]	train-logloss:0.01017	val-logloss:0.00667
[680]	train-logloss:0.00960	val-logloss:0.00662
[720]	train-logloss:0.00913	val-logloss:0.00661
[760]	train-logloss:0.00864	val-logloss:0.00650
[800]	train-logloss:0.00822	val-logloss:0.00

In [7]:
%%time
y_true = val[label].tolist()
y_pred = model.predict(data=dval, iteration_range=(0, model.best_iteration+1))
auc = roc_auc_score(y_true, y_pred, average="macro")
print(f"auc={auc:.4f}")
print(f"y_pred={y_pred.shape}\n{y_pred[:5]}")

auc=0.9943
y_pred=(2083,)
[0.00177922 0.8544107  0.78170127 0.46973562 0.20937915]
CPU times: user 789 ms, sys: 615 ms, total: 1.4 s
Wall time: 105 ms


In [8]:
%%time
scores = model.get_score(importance_type="gain")
assert len(scores)!=0
rows = []
for feature, score in scores.items():
    rows.append({'importance': score, 'feature': feature})
idf = pd.DataFrame.from_records(rows)
idf = idf.sort_values(["importance"], ascending=False, ignore_index=True)
fp = f"{job_dir}/importance.csv"
idf.to_csv(fp, index=True)
print(f"Saved {fp}")
idf.T.head()

Saved models/xgb/20240121_143035/importance.csv
CPU times: user 269 ms, sys: 266 ms, total: 536 ms
Wall time: 42.1 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886
importance,930.71875,728.540649,537.788269,464.965485,349.631927,325.706543,318.117737,316.712067,303.387024,290.701691,273.247772,266.315247,266.088867,260.14444,244.749756,230.208511,200.259109,187.807129,182.452652,176.945465,176.619553,163.965012,156.123428,154.240265,153.022537,150.847763,148.322372,144.387253,143.910019,143.608017,142.920547,138.298172,136.441833,134.898956,132.832748,132.8293,130.934494,127.713005,127.067108,122.888718,121.326965,121.132416,120.279121,109.100029,107.869934,101.994423,99.426315,98.646088,97.850769,97.749084,96.68782

In [9]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:29:28.528191
